In [1]:
import strax
import straxen
print(straxen.__file__)
print(straxen.__version__)
print(strax.__version__)
import pandas as pd
pd.options.display.max_colwidth = 100
from tqdm import tqdm
import numpy as np

import numba

from scipy.optimize import curve_fit
from scipy import stats

import matplotlib.pyplot as plt

/home/gvolta/XENONnT/straxen/straxen/__init__.py
0.5.0
0.8.6


In [6]:
strax.Mailbox.DEFAULT_MAX_MESSAGES = 2

In [7]:
st = straxen.contexts.strax_SPE()

In [8]:
st.show_config()

,option,default,current,applies_to,help
0,storage_converter,False,False,Context,"If True, save data that is loaded from one frontend through all willing other storage frontends."
1,fuzzy_for,(),(),Context,"Tuple of plugin names for which no checks for version, providing plugin, and config will be perf..."
2,fuzzy_for_options,(),(),Context,Tuple of config options for which no checks will be performed when looking for data.
3,allow_incomplete,False,False,Context,"Allow loading of incompletely written data, if the storage systems support it"
4,allow_rechunk,True,True,Context,Allow rechunking of data during writing.
5,allow_multiprocess,False,True,Context,"Allow multiprocessing.If False, will use multithreading only."
6,allow_shm,False,False,Context,Allow use of /dev/shm for interprocess communication.
7,forbid_creation_of,(),"(raw_records,)",Context,"If any of the following datatypes is requested to be created, throw an error instead. Useful to ..."
8,store_run_fields,(),"(name, number, reader.ini.name, tags.name, start, end, livetime, trigger.events_built)",Context,Tuple of run document fields to store during scan_run.
9,check_available,(),"(raw_records, event_info)",Context,Tuple of data types to scan availability for during scan_run.


In [9]:
st.show_config('led_calibration')

,option,default,current,applies_to,help
0,led_window,"(150, 275)",<OMITTED>,"(led_calibration,)",Window (samples) where we expect the signal in LED calibration
1,noise_window,"(0, 125)",<OMITTED>,"(led_calibration,)",Window (samples) to analysis the noise
2,channel_list,"(0, 248)",<OMITTED>,"(led_calibration,)","Three different light level for XENON1T: (0,36), (37,126), (127,248). Defalt value: all the PMTs"
3,pax_raw_dir,/data/xenon/raw,<OMITTED>,"(raw_records,)",Directory with raw pax datasets
4,stop_after_zips,0,<OMITTED>,"(raw_records,)",Convert only this many zip files. 0 = all.
5,events_per_chunk,50,<OMITTED>,"(raw_records,)",Number of events to yield per chunk
6,samples_per_record,110,<OMITTED>,"(raw_records,)",Number of samples per record


In [ ]:
st = st.new_context(config=dict(channel_list=(0,1)))
st.show_config('led_calibration')

In [ ]:
st.show_config('led_calibration')

In [ ]:
runs = st.select_runs(run_mode='LED*')

In [ ]:
runs

#### Hist of windows integration by raw_records plugin

In [ ]:
run_id = '180219_1049'
data_rr = st.get_array(run_id, 'raw_records', max_workers = 10, seconds_range=(0,5))

#### Hist of windows integration by led_calibration plugin

In [ ]:
data_lc = st.get_array('180219_1049', 'led_calibration', max_workers = 10)

In [ ]:
def get_amplitude(raw_records, led_window=[125, 250], noise_window=[350, 475]):
    '''
    Needed for the SPE computation.
    Take the maximum in two different regions, where there is the signal and where there is not.
    '''
    on = np.zeros((len(raw_records)), dtype=[('channel','int16'),('amplitude_led', '<i4')])
    off = np.zeros((len(raw_records)), dtype=[('channel','int16'),('amplitude_noise', '<i4')])
    i = 0
    for r in raw_records:
        on['amplitude_led'][i] = np.max(r['data'][led_window[0]:led_window[1]])
        on['channel'][i] = r['channel']
        off['amplitude_noise'][i] = np.max(r['data'][noise_window[0]:noise_window[1]])
        off['channel'][i] = r['channel']
        i=i+1
    return on, off

def get_area(raw_records, LED_window=[125, 250]):
    '''
    Needed for the gain computation.
    Sum the data in the defined window to get the area.
    This is done in 6 integration window and it returns the average area.
    '''
    left = LED_window[0]
    end_pos = [LED_window[1]+2*i for i in range(6)]

    Area = np.zeros((len(raw_records)), dtype=[('channel','int16'),('area','float32')])
    for right in end_pos:
        Area['area'] += raw_records['data'][:, left:right].sum(axis=1)
    Area['channel'] = raw_records['channel']
    Area['area'] = Area['area']/float(len(end_pos))
        
    return Area

In [ ]:
on, off = get_amplitude(data_rr)
Area = get_area(data_rr)

In [ ]:
data_lc

### Comparison

In [ ]:
plt.figure(figsize=(20,22))

plt.subplot(321)
plt.hist(data_lc[data_lc['channel']==0]['area'], bins=200, range=(-500, 2000), 
         color='red', histtype='step', density=True, label='lc')
plt.yscale('log')
plt.xlabel('Area')
#plt.title('led_calibration')

plt.subplot(322)
plt.hist(Area[Area['channel']==0]['area'], bins=200, range=(-500, 2000), 
         color='gold', histtype='step', density=True, label='rr')
plt.yscale('log')
plt.xlabel('Area')
#plt.title('raw_records')

plt.subplot(323)
plt.hist(data_lc[data_lc['channel']==0]['amplitude_led'], bins=100, range=(-5, 200), 
         color='red', histtype='step', density=True, label='lc')
plt.hist(data_lc[data_lc['channel']==0]['amplitude_noise'], bins=100, range=(-5, 200), 
         color='black', histtype='step', density=True, label='lc')
plt.yscale('log')
plt.xlabel('Amplitude')

plt.subplot(324)
plt.hist(on[on['channel']==0]['amplitude_led'], bins=100, range=(-5, 200), 
         color='gold', histtype='step', density=True, label='rr')
plt.hist(off[off['channel']==0]['amplitude_noise'], bins=100, range=(-5, 200), 
         color='black', histtype='step', density=True, label='rr')
plt.yscale('log')
#plt.xlabel('Amplitude')

plt.subplot(325)
plt.hist(data_lc[data_lc['channel']==0]['area'], bins=200, range=(-500, 2000), 
         color='red', histtype='step', density=True, label='lc')
plt.hist(Area[Area['channel']==0]['area'], bins=200, range=(-500, 2000), 
         color='gold', histtype='step', density=True, label='rr')
plt.yscale('log')
plt.xlabel('Area')
plt.legend(loc='best')
#plt.title('raw_records')

plt.subplot(326)
plt.hist(on[on['channel']==0]['amplitude_led'], bins=100, range=(-5, 200), 
         color='gold', histtype='step', density=True, label='lc')
plt.hist(data_lc[data_lc['channel']==0]['amplitude_led'], bins=100, range=(-5, 200), 
         color='red', histtype='step', density=True, label='rr')
plt.yscale('log')
plt.xlabel('Amplitude')
plt.legend(loc='best')

plt.show()
plt.tight_layout()